# **Design a Text Retrieval System**

In [1]:
# Imports
from google.colab import drive
from pathlib import Path

import os
import pandas as pd                        # Library used for loading dataset


In [2]:
# Setting up the notebook.

# 1. Mounting the drive
drive.mount('/content/gdrive', force_remount=True)

# 2. Path on the drive to work in
base_path = Path("/content/gdrive/MyDrive/Assessment 03 - Data folder-20240525")


Mounted at /content/gdrive


**Data from files**

In [3]:
train_data = pd.read_csv(base_path.joinpath("train.csv"),encoding="ISO-8859-1")     # Loaded training data from train.csv file to train_data

In [4]:
test_data = pd.read_csv(base_path.joinpath("test.csv"),encoding="ISO-8859-1")       # Loaded test data from test.csv file to test_data

In [5]:
test_data = test_data.dropna()
train_data = train_data.dropna()

In [6]:
# Remove duplicate rows based on both title and description columns
train_data = train_data.drop_duplicates(subset=['title', 'description'])
test_data = test_data.drop_duplicates(subset=['title', 'description'])



In [7]:
unique_titles_count = train_data['title'].nunique()

if unique_titles_count == len(train_data):
    print("All titles are unique.")
else:
    print(f"There are {len(train_data) - unique_titles_count} duplicate titles.")

There are 74 duplicate titles.


In [8]:
title_counts = train_data['title'].value_counts()

max_repeated_count = title_counts.max()

print(f"The maximum number of times a title is repeated is: {max_repeated_count}")
print(title_counts)
print(unique_titles_count)

The maximum number of times a title is repeated is: 2
title
The Tunnel                                                2
Friends with Benefits                                     2
Blood Ties                                                2
Life                                                      2
Waking the Dead                                           2
                                                         ..
Imaginary Heroes                                          1
Licorice Pizza                                            1
Pirates of the Caribbean: The Curse of the Black Pearl    1
Ender's Game                                              1
Ne Zha                                                    1
Name: count, Length: 5968, dtype: int64
5968


In [9]:
# Step 1: Initialize lists for train_data and val_data
train_list = []
val_list = []
title_tracker = {}

# Step 2: Iterate through the DataFrame
for index, row in train_data.iterrows():
    title = row['title']
    if title in title_tracker:
        # If the title is already in title_tracker, add this row to val_list
        val_list.append(row)
    else:
        # If the title is not in title_tracker, add this row to train_list and mark it as added
        train_list.append(row)
        title_tracker[title] = True

# Convert lists back to DataFrames
train_data = pd.DataFrame(train_list)
val_data = pd.DataFrame(val_list)

# Resetting index for both datasets
train_data.reset_index(drop=True, inplace=True)
val_data.reset_index(drop=True, inplace=True)

# Output the results
print("Training Data:")
print(train_data)
print("\nValidation Data:")
print(val_data)

Training Data:
                           title  \
0                         Tsotsi   
1        Abducted in Plain Sight   
2              My Life Is Murder   
3                         Empire   
4                    Latter Days   
...                          ...   
5963  Team America: World Police   
5964     Shaadi Mein Zaroor Aana   
5965           Inhuman Resources   
5966               Hemlock Grove   
5967                      Ne Zha   

                                            description  
0     A South African hoodlum named Tsotsi (Presley ...  
1     In 1974 a 12-year-old girl is abducted from a ...  
2     Private investigator Alexa Crow always fights ...  
3     Hip-hop artist and CEO of Empire Entertainment...  
4     Aaron Davis (Steve Sandvoss), a young Mormon, ...  
...                                                 ...  
5963  When North Korean ruler Kim Jong-il (Trey Park...  
5964  Aarti, a bright, ambitious woman, is on the cu...  
5965  Unemployed and desperate

In [10]:
print(train_data[train_data['title']== 'The Fall']['description'])

322    A statement on violence, revolution and the tu...
Name: description, dtype: object


In [11]:
#expanding the dispay of text description column
pd.set_option('display.max_colwidth', 150)
#using only title and description column
data= train_data [['title','description']]
data.head()

,title,description
0,Tsotsi,"A South African hoodlum named Tsotsi (Presley Chweneyagae) lives by a code of violence, and he and his gang of thugs prowl the streets of Johannes..."
1,Abducted in Plain Sight,In 1974 a 12-year-old girl is abducted from a small church-going community in Idaho.
2,My Life Is Murder,"Private investigator Alexa Crow always fights the good fight, whether it be a baffling crime or just in her everyday life."
3,Empire,"Hip-hop artist and CEO of Empire Entertainment, Lucious Lyon, has always ruled unchallenged, but a medical diagnosis predicts he will be incapacit..."
4,Latter Days,"Aaron Davis (Steve Sandvoss), a young Mormon, arrives in Los Angeles with three fellow missionaries to evangelize. The group's promiscuous gay nei..."


In [12]:
data.shape

(5968, 2)

In [13]:
#expanding the dispay of text description column
pd.set_option('display.max_colwidth', 100)
#using only title and description column
val_data= val_data [['title','description']]
val_data.head(15)

,title,description
0,The Fall,The psychological thriller examines the lives of two hunters -- one is a serial killer who preys...
1,First Love,A psychologist decides to write a book on a university student who suddenly gave up her dreams t...
2,The Vow,"Following the experiences of people deeply involved in the self-improvement group NXIVM, an orga..."
3,Resident Evil,"Year 2036 -- 14 years after the spread of Joy caused so much pain, Jade Wesker fights for surviv..."
4,Constantine,Demon hunter and master of the occult John Constantine figured there was no longer any point in ...
5,Taken,"Bryan Mills (Liam Neeson), a former government operative, is trying to reconnect with his daught..."
6,Black Dynamite,"After ""The Man"" kills his brother and poisons the neighborhood with tainted liquor, a kung fu fi..."
7,Broken,"Father Michael Kerrigan, a modern, but maverick, Catholic priest, juggles parochial problems wit..."
8,Lucifer,A mad scramble for power ensues when the leader of a political party dies suddenly.
9,Inside,A couple confine a young man (Nicholas D'Agosto) who resembles their late son.


In [14]:
val_data.shape

(74, 2)

In [15]:
#expanding the dispay of text description column
pd.set_option('display.max_colwidth', 100)
#using only title and description column
test_data= test_data [['title','description']]
test_data.head(15)

,title,description
0,Splitting Up Together,Ellen DeGeneres serves as executive producer of this comedy that is based on a Danish series of ...
1,The Trip,A collage of film images and ambient dance sounds from Jacques Peretti and DJ Downfall.
2,The Kids Are Alright,"Set in the 1970s, this ensemble comedy follows a traditional Irish-Catholic family, the Clearys,..."
3,Breeders,"""Breeders"" explores the paradox experienced by nearly all parents, the willingness to die for on..."
4,Let the Right One In,"The lives of Mark and his daughter, Eleanor, were forever changed 10 years earlier when she was ..."
5,Raja Rani,"On the run from police, a thief (Rajesh Khanna) crashes a wedding and winds up marrying the brid..."
6,Bommarillu,"Siddhu's overprotective father decides to get him married to a rich girl. Siddhu's life, however..."
7,Father Brown,Based on the novels of G.K. Chesterton.
8,Jane the Virgin,"Jane Villanueva, the daughter of a teen mother, grew up determined not to repeat her mom's mista..."
9,Morvern Callar,"When her boyfriend commits suicide, unambitious Glasgow clerk Morvern Callar (Samantha Morton) i..."


In [16]:
test_data.shape

(10, 2)

# **Data pre-processing**

# Punctuation Removal


In [17]:
#library that contains punctuation
import string
#defining the function to remove punctuation
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
#storing the puntuation free text
data['clean_msg']= data['description'].apply(lambda x:remove_punctuation(x))
data.head()

,title,description,clean_msg
0,Tsotsi,"A South African hoodlum named Tsotsi (Presley Chweneyagae) lives by a code of violence, and he a...",A South African hoodlum named Tsotsi Presley Chweneyagae lives by a code of violence and he and ...
1,Abducted in Plain Sight,In 1974 a 12-year-old girl is abducted from a small church-going community in Idaho.,In 1974 a 12yearold girl is abducted from a small churchgoing community in Idaho
2,My Life Is Murder,"Private investigator Alexa Crow always fights the good fight, whether it be a baffling crime or ...",Private investigator Alexa Crow always fights the good fight whether it be a baffling crime or j...
3,Empire,"Hip-hop artist and CEO of Empire Entertainment, Lucious Lyon, has always ruled unchallenged, but...",Hiphop artist and CEO of Empire Entertainment Lucious Lyon has always ruled unchallenged but a m...
4,Latter Days,"Aaron Davis (Steve Sandvoss), a young Mormon, arrives in Los Angeles with three fellow missionar...",Aaron Davis Steve Sandvoss a young Mormon arrives in Los Angeles with three fellow missionaries ...


In [18]:
val_data['clean_msg']= val_data['description'].apply(lambda x:remove_punctuation(x))
val_data.head()

,title,description,clean_msg
0,The Fall,The psychological thriller examines the lives of two hunters -- one is a serial killer who preys...,The psychological thriller examines the lives of two hunters one is a serial killer who preys o...
1,First Love,A psychologist decides to write a book on a university student who suddenly gave up her dreams t...,A psychologist decides to write a book on a university student who suddenly gave up her dreams t...
2,The Vow,"Following the experiences of people deeply involved in the self-improvement group NXIVM, an orga...",Following the experiences of people deeply involved in the selfimprovement group NXIVM an organi...
3,Resident Evil,"Year 2036 -- 14 years after the spread of Joy caused so much pain, Jade Wesker fights for surviv...",Year 2036 14 years after the spread of Joy caused so much pain Jade Wesker fights for survival ...
4,Constantine,Demon hunter and master of the occult John Constantine figured there was no longer any point in ...,Demon hunter and master of the occult John Constantine figured there was no longer any point in ...


In [19]:
test_data['clean_msg']= test_data['description'].apply(lambda x:remove_punctuation(x))
test_data.head()

,title,description,clean_msg
0,Splitting Up Together,Ellen DeGeneres serves as executive producer of this comedy that is based on a Danish series of ...,Ellen DeGeneres serves as executive producer of this comedy that is based on a Danish series of ...
1,The Trip,A collage of film images and ambient dance sounds from Jacques Peretti and DJ Downfall.,A collage of film images and ambient dance sounds from Jacques Peretti and DJ Downfall
2,The Kids Are Alright,"Set in the 1970s, this ensemble comedy follows a traditional Irish-Catholic family, the Clearys,...",Set in the 1970s this ensemble comedy follows a traditional IrishCatholic family the Clearys as ...
3,Breeders,"""Breeders"" explores the paradox experienced by nearly all parents, the willingness to die for on...",Breeders explores the paradox experienced by nearly all parents the willingness to die for ones ...
4,Let the Right One In,"The lives of Mark and his daughter, Eleanor, were forever changed 10 years earlier when she was ...",The lives of Mark and his daughter Eleanor were forever changed 10 years earlier when she was tu...


# Lowering the Text


In [20]:
data['msg_lower']= data['clean_msg'].apply(lambda x: x.lower())
data.head()

,title,description,clean_msg,msg_lower
0,Tsotsi,"A South African hoodlum named Tsotsi (Presley Chweneyagae) lives by a code of violence, and he a...",A South African hoodlum named Tsotsi Presley Chweneyagae lives by a code of violence and he and ...,a south african hoodlum named tsotsi presley chweneyagae lives by a code of violence and he and ...
1,Abducted in Plain Sight,In 1974 a 12-year-old girl is abducted from a small church-going community in Idaho.,In 1974 a 12yearold girl is abducted from a small churchgoing community in Idaho,in 1974 a 12yearold girl is abducted from a small churchgoing community in idaho
2,My Life Is Murder,"Private investigator Alexa Crow always fights the good fight, whether it be a baffling crime or ...",Private investigator Alexa Crow always fights the good fight whether it be a baffling crime or j...,private investigator alexa crow always fights the good fight whether it be a baffling crime or j...
3,Empire,"Hip-hop artist and CEO of Empire Entertainment, Lucious Lyon, has always ruled unchallenged, but...",Hiphop artist and CEO of Empire Entertainment Lucious Lyon has always ruled unchallenged but a m...,hiphop artist and ceo of empire entertainment lucious lyon has always ruled unchallenged but a m...
4,Latter Days,"Aaron Davis (Steve Sandvoss), a young Mormon, arrives in Los Angeles with three fellow missionar...",Aaron Davis Steve Sandvoss a young Mormon arrives in Los Angeles with three fellow missionaries ...,aaron davis steve sandvoss a young mormon arrives in los angeles with three fellow missionaries ...


In [21]:
val_data['msg_lower']= val_data['clean_msg'].apply(lambda x: x.lower())
val_data.head()

,title,description,clean_msg,msg_lower
0,The Fall,The psychological thriller examines the lives of two hunters -- one is a serial killer who preys...,The psychological thriller examines the lives of two hunters one is a serial killer who preys o...,the psychological thriller examines the lives of two hunters one is a serial killer who preys o...
1,First Love,A psychologist decides to write a book on a university student who suddenly gave up her dreams t...,A psychologist decides to write a book on a university student who suddenly gave up her dreams t...,a psychologist decides to write a book on a university student who suddenly gave up her dreams t...
2,The Vow,"Following the experiences of people deeply involved in the self-improvement group NXIVM, an orga...",Following the experiences of people deeply involved in the selfimprovement group NXIVM an organi...,following the experiences of people deeply involved in the selfimprovement group nxivm an organi...
3,Resident Evil,"Year 2036 -- 14 years after the spread of Joy caused so much pain, Jade Wesker fights for surviv...",Year 2036 14 years after the spread of Joy caused so much pain Jade Wesker fights for survival ...,year 2036 14 years after the spread of joy caused so much pain jade wesker fights for survival ...
4,Constantine,Demon hunter and master of the occult John Constantine figured there was no longer any point in ...,Demon hunter and master of the occult John Constantine figured there was no longer any point in ...,demon hunter and master of the occult john constantine figured there was no longer any point in ...


In [22]:
test_data['msg_lower']= test_data['clean_msg'].apply(lambda x: x.lower())
test_data.head()

,title,description,clean_msg,msg_lower
0,Splitting Up Together,Ellen DeGeneres serves as executive producer of this comedy that is based on a Danish series of ...,Ellen DeGeneres serves as executive producer of this comedy that is based on a Danish series of ...,ellen degeneres serves as executive producer of this comedy that is based on a danish series of ...
1,The Trip,A collage of film images and ambient dance sounds from Jacques Peretti and DJ Downfall.,A collage of film images and ambient dance sounds from Jacques Peretti and DJ Downfall,a collage of film images and ambient dance sounds from jacques peretti and dj downfall
2,The Kids Are Alright,"Set in the 1970s, this ensemble comedy follows a traditional Irish-Catholic family, the Clearys,...",Set in the 1970s this ensemble comedy follows a traditional IrishCatholic family the Clearys as ...,set in the 1970s this ensemble comedy follows a traditional irishcatholic family the clearys as ...
3,Breeders,"""Breeders"" explores the paradox experienced by nearly all parents, the willingness to die for on...",Breeders explores the paradox experienced by nearly all parents the willingness to die for ones ...,breeders explores the paradox experienced by nearly all parents the willingness to die for ones ...
4,Let the Right One In,"The lives of Mark and his daughter, Eleanor, were forever changed 10 years earlier when she was ...",The lives of Mark and his daughter Eleanor were forever changed 10 years earlier when she was tu...,the lives of mark and his daughter eleanor were forever changed 10 years earlier when she was tu...


# Tokenisation


In [23]:
#defining function for tokenisation
import re
def tokenisation(text):
    tokens = re.split('\W+',text)
    return tokens
#applying function to the column
data['msg_tokenied']= data['msg_lower'].apply(lambda x: tokenisation(x))
data.head()

,title,description,clean_msg,msg_lower,msg_tokenied
0,Tsotsi,"A South African hoodlum named Tsotsi (Presley Chweneyagae) lives by a code of violence, and he a...",A South African hoodlum named Tsotsi Presley Chweneyagae lives by a code of violence and he and ...,a south african hoodlum named tsotsi presley chweneyagae lives by a code of violence and he and ...,"[a, south, african, hoodlum, named, tsotsi, presley, chweneyagae, lives, by, a, code, of, violen..."
1,Abducted in Plain Sight,In 1974 a 12-year-old girl is abducted from a small church-going community in Idaho.,In 1974 a 12yearold girl is abducted from a small churchgoing community in Idaho,in 1974 a 12yearold girl is abducted from a small churchgoing community in idaho,"[in, 1974, a, 12yearold, girl, is, abducted, from, a, small, churchgoing, community, in, idaho]"
2,My Life Is Murder,"Private investigator Alexa Crow always fights the good fight, whether it be a baffling crime or ...",Private investigator Alexa Crow always fights the good fight whether it be a baffling crime or j...,private investigator alexa crow always fights the good fight whether it be a baffling crime or j...,"[private, investigator, alexa, crow, always, fights, the, good, fight, whether, it, be, a, baffl..."
3,Empire,"Hip-hop artist and CEO of Empire Entertainment, Lucious Lyon, has always ruled unchallenged, but...",Hiphop artist and CEO of Empire Entertainment Lucious Lyon has always ruled unchallenged but a m...,hiphop artist and ceo of empire entertainment lucious lyon has always ruled unchallenged but a m...,"[hiphop, artist, and, ceo, of, empire, entertainment, lucious, lyon, has, always, ruled, unchall..."
4,Latter Days,"Aaron Davis (Steve Sandvoss), a young Mormon, arrives in Los Angeles with three fellow missionar...",Aaron Davis Steve Sandvoss a young Mormon arrives in Los Angeles with three fellow missionaries ...,aaron davis steve sandvoss a young mormon arrives in los angeles with three fellow missionaries ...,"[aaron, davis, steve, sandvoss, a, young, mormon, arrives, in, los, angeles, with, three, fellow..."


In [24]:
val_data['msg_tokenied']= val_data['msg_lower'].apply(lambda x: tokenisation(x))
val_data.head()

,title,description,clean_msg,msg_lower,msg_tokenied
0,The Fall,The psychological thriller examines the lives of two hunters -- one is a serial killer who preys...,The psychological thriller examines the lives of two hunters one is a serial killer who preys o...,the psychological thriller examines the lives of two hunters one is a serial killer who preys o...,"[the, psychological, thriller, examines, the, lives, of, two, hunters, one, is, a, serial, kille..."
1,First Love,A psychologist decides to write a book on a university student who suddenly gave up her dreams t...,A psychologist decides to write a book on a university student who suddenly gave up her dreams t...,a psychologist decides to write a book on a university student who suddenly gave up her dreams t...,"[a, psychologist, decides, to, write, a, book, on, a, university, student, who, suddenly, gave, ..."
2,The Vow,"Following the experiences of people deeply involved in the self-improvement group NXIVM, an orga...",Following the experiences of people deeply involved in the selfimprovement group NXIVM an organi...,following the experiences of people deeply involved in the selfimprovement group nxivm an organi...,"[following, the, experiences, of, people, deeply, involved, in, the, selfimprovement, group, nxi..."
3,Resident Evil,"Year 2036 -- 14 years after the spread of Joy caused so much pain, Jade Wesker fights for surviv...",Year 2036 14 years after the spread of Joy caused so much pain Jade Wesker fights for survival ...,year 2036 14 years after the spread of joy caused so much pain jade wesker fights for survival ...,"[year, 2036, 14, years, after, the, spread, of, joy, caused, so, much, pain, jade, wesker, fight..."
4,Constantine,Demon hunter and master of the occult John Constantine figured there was no longer any point in ...,Demon hunter and master of the occult John Constantine figured there was no longer any point in ...,demon hunter and master of the occult john constantine figured there was no longer any point in ...,"[demon, hunter, and, master, of, the, occult, john, constantine, figured, there, was, no, longer..."


In [25]:
test_data['msg_tokenied']= test_data['msg_lower'].apply(lambda x: tokenisation(x))
test_data.head()

,title,description,clean_msg,msg_lower,msg_tokenied
0,Splitting Up Together,Ellen DeGeneres serves as executive producer of this comedy that is based on a Danish series of ...,Ellen DeGeneres serves as executive producer of this comedy that is based on a Danish series of ...,ellen degeneres serves as executive producer of this comedy that is based on a danish series of ...,"[ellen, degeneres, serves, as, executive, producer, of, this, comedy, that, is, based, on, a, da..."
1,The Trip,A collage of film images and ambient dance sounds from Jacques Peretti and DJ Downfall.,A collage of film images and ambient dance sounds from Jacques Peretti and DJ Downfall,a collage of film images and ambient dance sounds from jacques peretti and dj downfall,"[a, collage, of, film, images, and, ambient, dance, sounds, from, jacques, peretti, and, dj, dow..."
2,The Kids Are Alright,"Set in the 1970s, this ensemble comedy follows a traditional Irish-Catholic family, the Clearys,...",Set in the 1970s this ensemble comedy follows a traditional IrishCatholic family the Clearys as ...,set in the 1970s this ensemble comedy follows a traditional irishcatholic family the clearys as ...,"[set, in, the, 1970s, this, ensemble, comedy, follows, a, traditional, irishcatholic, family, th..."
3,Breeders,"""Breeders"" explores the paradox experienced by nearly all parents, the willingness to die for on...",Breeders explores the paradox experienced by nearly all parents the willingness to die for ones ...,breeders explores the paradox experienced by nearly all parents the willingness to die for ones ...,"[breeders, explores, the, paradox, experienced, by, nearly, all, parents, the, willingness, to, ..."
4,Let the Right One In,"The lives of Mark and his daughter, Eleanor, were forever changed 10 years earlier when she was ...",The lives of Mark and his daughter Eleanor were forever changed 10 years earlier when she was tu...,the lives of mark and his daughter eleanor were forever changed 10 years earlier when she was tu...,"[the, lives, of, mark, and, his, daughter, eleanor, were, forever, changed, 10, years, earlier, ..."


# Stop Word Removal


In [26]:
#importing nlp library
import nltk
#Stop words present in the library
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
print(stopwords[:10])


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [27]:
#defining the function to remove stopwords from tokenized text
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output
#applying the function
data['no_stopwords']= data['msg_tokenied'].apply(lambda x:remove_stopwords(x))
data.head()

,title,description,clean_msg,msg_lower,msg_tokenied,no_stopwords
0,Tsotsi,"A South African hoodlum named Tsotsi (Presley Chweneyagae) lives by a code of violence, and he a...",A South African hoodlum named Tsotsi Presley Chweneyagae lives by a code of violence and he and ...,a south african hoodlum named tsotsi presley chweneyagae lives by a code of violence and he and ...,"[a, south, african, hoodlum, named, tsotsi, presley, chweneyagae, lives, by, a, code, of, violen...","[south, african, hoodlum, named, tsotsi, presley, chweneyagae, lives, code, violence, gang, thug..."
1,Abducted in Plain Sight,In 1974 a 12-year-old girl is abducted from a small church-going community in Idaho.,In 1974 a 12yearold girl is abducted from a small churchgoing community in Idaho,in 1974 a 12yearold girl is abducted from a small churchgoing community in idaho,"[in, 1974, a, 12yearold, girl, is, abducted, from, a, small, churchgoing, community, in, idaho]","[1974, 12yearold, girl, abducted, small, churchgoing, community, idaho]"
2,My Life Is Murder,"Private investigator Alexa Crow always fights the good fight, whether it be a baffling crime or ...",Private investigator Alexa Crow always fights the good fight whether it be a baffling crime or j...,private investigator alexa crow always fights the good fight whether it be a baffling crime or j...,"[private, investigator, alexa, crow, always, fights, the, good, fight, whether, it, be, a, baffl...","[private, investigator, alexa, crow, always, fights, good, fight, whether, baffling, crime, ever..."
3,Empire,"Hip-hop artist and CEO of Empire Entertainment, Lucious Lyon, has always ruled unchallenged, but...",Hiphop artist and CEO of Empire Entertainment Lucious Lyon has always ruled unchallenged but a m...,hiphop artist and ceo of empire entertainment lucious lyon has always ruled unchallenged but a m...,"[hiphop, artist, and, ceo, of, empire, entertainment, lucious, lyon, has, always, ruled, unchall...","[hiphop, artist, ceo, empire, entertainment, lucious, lyon, always, ruled, unchallenged, medical..."
4,Latter Days,"Aaron Davis (Steve Sandvoss), a young Mormon, arrives in Los Angeles with three fellow missionar...",Aaron Davis Steve Sandvoss a young Mormon arrives in Los Angeles with three fellow missionaries ...,aaron davis steve sandvoss a young mormon arrives in los angeles with three fellow missionaries ...,"[aaron, davis, steve, sandvoss, a, young, mormon, arrives, in, los, angeles, with, three, fellow...","[aaron, davis, steve, sandvoss, young, mormon, arrives, los, angeles, three, fellow, missionarie..."


In [28]:
val_data['no_stopwords']= val_data['msg_tokenied'].apply(lambda x:remove_stopwords(x))
val_data.head()

,title,description,clean_msg,msg_lower,msg_tokenied,no_stopwords
0,The Fall,The psychological thriller examines the lives of two hunters -- one is a serial killer who preys...,The psychological thriller examines the lives of two hunters one is a serial killer who preys o...,the psychological thriller examines the lives of two hunters one is a serial killer who preys o...,"[the, psychological, thriller, examines, the, lives, of, two, hunters, one, is, a, serial, kille...","[psychological, thriller, examines, lives, two, hunters, one, serial, killer, preys, victims, ar..."
1,First Love,A psychologist decides to write a book on a university student who suddenly gave up her dreams t...,A psychologist decides to write a book on a university student who suddenly gave up her dreams t...,a psychologist decides to write a book on a university student who suddenly gave up her dreams t...,"[a, psychologist, decides, to, write, a, book, on, a, university, student, who, suddenly, gave, ...","[psychologist, decides, write, book, university, student, suddenly, gave, dreams, become, announ..."
2,The Vow,"Following the experiences of people deeply involved in the self-improvement group NXIVM, an orga...",Following the experiences of people deeply involved in the selfimprovement group NXIVM an organi...,following the experiences of people deeply involved in the selfimprovement group nxivm an organi...,"[following, the, experiences, of, people, deeply, involved, in, the, selfimprovement, group, nxi...","[following, experiences, people, deeply, involved, selfimprovement, group, nxivm, organization, ..."
3,Resident Evil,"Year 2036 -- 14 years after the spread of Joy caused so much pain, Jade Wesker fights for surviv...",Year 2036 14 years after the spread of Joy caused so much pain Jade Wesker fights for survival ...,year 2036 14 years after the spread of joy caused so much pain jade wesker fights for survival ...,"[year, 2036, 14, years, after, the, spread, of, joy, caused, so, much, pain, jade, wesker, fight...","[year, 2036, 14, years, spread, joy, caused, much, pain, jade, wesker, fights, survival, world, ..."
4,Constantine,Demon hunter and master of the occult John Constantine figured there was no longer any point in ...,Demon hunter and master of the occult John Constantine figured there was no longer any point in ...,demon hunter and master of the occult john constantine figured there was no longer any point in ...,"[demon, hunter, and, master, of, the, occult, john, constantine, figured, there, was, no, longer...","[demon, hunter, master, occult, john, constantine, figured, longer, point, fighting, good, fight..."


In [29]:
test_data['no_stopwords']= test_data['msg_tokenied'].apply(lambda x:remove_stopwords(x))
test_data.head()

,title,description,clean_msg,msg_lower,msg_tokenied,no_stopwords
0,Splitting Up Together,Ellen DeGeneres serves as executive producer of this comedy that is based on a Danish series of ...,Ellen DeGeneres serves as executive producer of this comedy that is based on a Danish series of ...,ellen degeneres serves as executive producer of this comedy that is based on a danish series of ...,"[ellen, degeneres, serves, as, executive, producer, of, this, comedy, that, is, based, on, a, da...","[ellen, degeneres, serves, executive, producer, comedy, based, danish, series, name, lena, marti..."
1,The Trip,A collage of film images and ambient dance sounds from Jacques Peretti and DJ Downfall.,A collage of film images and ambient dance sounds from Jacques Peretti and DJ Downfall,a collage of film images and ambient dance sounds from jacques peretti and dj downfall,"[a, collage, of, film, images, and, ambient, dance, sounds, from, jacques, peretti, and, dj, dow...","[collage, film, images, ambient, dance, sounds, jacques, peretti, dj, downfall]"
2,The Kids Are Alright,"Set in the 1970s, this ensemble comedy follows a traditional Irish-Catholic family, the Clearys,...",Set in the 1970s this ensemble comedy follows a traditional IrishCatholic family the Clearys as ...,set in the 1970s this ensemble comedy follows a traditional irishcatholic family the clearys as ...,"[set, in, the, 1970s, this, ensemble, comedy, follows, a, traditional, irishcatholic, family, th...","[set, 1970s, ensemble, comedy, follows, traditional, irishcatholic, family, clearys, navigates, ..."
3,Breeders,"""Breeders"" explores the paradox experienced by nearly all parents, the willingness to die for on...",Breeders explores the paradox experienced by nearly all parents the willingness to die for ones ...,breeders explores the paradox experienced by nearly all parents the willingness to die for ones ...,"[breeders, explores, the, paradox, experienced, by, nearly, all, parents, the, willingness, to, ...","[breeders, explores, paradox, experienced, nearly, parents, willingness, die, ones, children, co..."
4,Let the Right One In,"The lives of Mark and his daughter, Eleanor, were forever changed 10 years earlier when she was ...",The lives of Mark and his daughter Eleanor were forever changed 10 years earlier when she was tu...,the lives of mark and his daughter eleanor were forever changed 10 years earlier when she was tu...,"[the, lives, of, mark, and, his, daughter, eleanor, were, forever, changed, 10, years, earlier, ...","[lives, mark, daughter, eleanor, forever, changed, 10, years, earlier, turned, vampire, eleanor,..."


# Stemming


In [30]:
#importing the Stemming function from nltk library
from nltk.stem.porter import PorterStemmer
#defining the object for stemming
porter_stemmer = PorterStemmer()
#defining a function for stemming
def stemming(text):
    stem_text = [porter_stemmer.stem(word) for word in text]
    return stem_text
data['msg_stemmed']=data['no_stopwords'].apply(lambda x: stemming(x))
data.head()

,title,description,clean_msg,msg_lower,msg_tokenied,no_stopwords,msg_stemmed
0,Tsotsi,"A South African hoodlum named Tsotsi (Presley Chweneyagae) lives by a code of violence, and he a...",A South African hoodlum named Tsotsi Presley Chweneyagae lives by a code of violence and he and ...,a south african hoodlum named tsotsi presley chweneyagae lives by a code of violence and he and ...,"[a, south, african, hoodlum, named, tsotsi, presley, chweneyagae, lives, by, a, code, of, violen...","[south, african, hoodlum, named, tsotsi, presley, chweneyagae, lives, code, violence, gang, thug...","[south, african, hoodlum, name, tsotsi, presley, chweneyaga, live, code, violenc, gang, thug, pr..."
1,Abducted in Plain Sight,In 1974 a 12-year-old girl is abducted from a small church-going community in Idaho.,In 1974 a 12yearold girl is abducted from a small churchgoing community in Idaho,in 1974 a 12yearold girl is abducted from a small churchgoing community in idaho,"[in, 1974, a, 12yearold, girl, is, abducted, from, a, small, churchgoing, community, in, idaho]","[1974, 12yearold, girl, abducted, small, churchgoing, community, idaho]","[1974, 12yearold, girl, abduct, small, churchgo, commun, idaho]"
2,My Life Is Murder,"Private investigator Alexa Crow always fights the good fight, whether it be a baffling crime or ...",Private investigator Alexa Crow always fights the good fight whether it be a baffling crime or j...,private investigator alexa crow always fights the good fight whether it be a baffling crime or j...,"[private, investigator, alexa, crow, always, fights, the, good, fight, whether, it, be, a, baffl...","[private, investigator, alexa, crow, always, fights, good, fight, whether, baffling, crime, ever...","[privat, investig, alexa, crow, alway, fight, good, fight, whether, baffl, crime, everyday, life]"
3,Empire,"Hip-hop artist and CEO of Empire Entertainment, Lucious Lyon, has always ruled unchallenged, but...",Hiphop artist and CEO of Empire Entertainment Lucious Lyon has always ruled unchallenged but a m...,hiphop artist and ceo of empire entertainment lucious lyon has always ruled unchallenged but a m...,"[hiphop, artist, and, ceo, of, empire, entertainment, lucious, lyon, has, always, ruled, unchall...","[hiphop, artist, ceo, empire, entertainment, lucious, lyon, always, ruled, unchallenged, medical...","[hiphop, artist, ceo, empir, entertain, luciou, lyon, alway, rule, unchalleng, medic, diagnosi, ..."
4,Latter Days,"Aaron Davis (Steve Sandvoss), a young Mormon, arrives in Los Angeles with three fellow missionar...",Aaron Davis Steve Sandvoss a young Mormon arrives in Los Angeles with three fellow missionaries ...,aaron davis steve sandvoss a young mormon arrives in los angeles with three fellow missionaries ...,"[aaron, davis, steve, sandvoss, a, young, mormon, arrives, in, los, angeles, with, three, fellow...","[aaron, davis, steve, sandvoss, young, mormon, arrives, los, angeles, three, fellow, missionarie...","[aaron, davi, steve, sandvoss, young, mormon, arriv, lo, angel, three, fellow, missionari, evang..."


# Lemmatisation


In [31]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
#defining the object for lemmatisation
wordnet_lemmatizer = WordNetLemmatizer()
#defining the function for lemmatisation
def lemmatiser(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text
data['msg_lemmatized']=data['no_stopwords'].apply(lambda x:lemmatiser(x))
data.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...


,title,description,clean_msg,msg_lower,msg_tokenied,no_stopwords,msg_stemmed,msg_lemmatized
0,Tsotsi,"A South African hoodlum named Tsotsi (Presley Chweneyagae) lives by a code of violence, and he a...",A South African hoodlum named Tsotsi Presley Chweneyagae lives by a code of violence and he and ...,a south african hoodlum named tsotsi presley chweneyagae lives by a code of violence and he and ...,"[a, south, african, hoodlum, named, tsotsi, presley, chweneyagae, lives, by, a, code, of, violen...","[south, african, hoodlum, named, tsotsi, presley, chweneyagae, lives, code, violence, gang, thug...","[south, african, hoodlum, name, tsotsi, presley, chweneyaga, live, code, violenc, gang, thug, pr...","[south, african, hoodlum, named, tsotsi, presley, chweneyagae, life, code, violence, gang, thug,..."
1,Abducted in Plain Sight,In 1974 a 12-year-old girl is abducted from a small church-going community in Idaho.,In 1974 a 12yearold girl is abducted from a small churchgoing community in Idaho,in 1974 a 12yearold girl is abducted from a small churchgoing community in idaho,"[in, 1974, a, 12yearold, girl, is, abducted, from, a, small, churchgoing, community, in, idaho]","[1974, 12yearold, girl, abducted, small, churchgoing, community, idaho]","[1974, 12yearold, girl, abduct, small, churchgo, commun, idaho]","[1974, 12yearold, girl, abducted, small, churchgoing, community, idaho]"
2,My Life Is Murder,"Private investigator Alexa Crow always fights the good fight, whether it be a baffling crime or ...",Private investigator Alexa Crow always fights the good fight whether it be a baffling crime or j...,private investigator alexa crow always fights the good fight whether it be a baffling crime or j...,"[private, investigator, alexa, crow, always, fights, the, good, fight, whether, it, be, a, baffl...","[private, investigator, alexa, crow, always, fights, good, fight, whether, baffling, crime, ever...","[privat, investig, alexa, crow, alway, fight, good, fight, whether, baffl, crime, everyday, life]","[private, investigator, alexa, crow, always, fight, good, fight, whether, baffling, crime, every..."
3,Empire,"Hip-hop artist and CEO of Empire Entertainment, Lucious Lyon, has always ruled unchallenged, but...",Hiphop artist and CEO of Empire Entertainment Lucious Lyon has always ruled unchallenged but a m...,hiphop artist and ceo of empire entertainment lucious lyon has always ruled unchallenged but a m...,"[hiphop, artist, and, ceo, of, empire, entertainment, lucious, lyon, has, always, ruled, unchall...","[hiphop, artist, ceo, empire, entertainment, lucious, lyon, always, ruled, unchallenged, medical...","[hiphop, artist, ceo, empir, entertain, luciou, lyon, alway, rule, unchalleng, medic, diagnosi, ...","[hiphop, artist, ceo, empire, entertainment, lucious, lyon, always, ruled, unchallenged, medical..."
4,Latter Days,"Aaron Davis (Steve Sandvoss), a young Mormon, arrives in Los Angeles with three fellow missionar...",Aaron Davis Steve Sandvoss a young Mormon arrives in Los Angeles with three fellow missionaries ...,aaron davis steve sandvoss a young mormon arrives in los angeles with three fellow missionaries ...,"[aaron, davis, steve, sandvoss, a, young, mormon, arrives, in, los, angeles, with, three, fellow...","[aaron, davis, steve, sandvoss, young, mormon, arrives, los, angeles, three, fellow, missionarie...","[aaron, davi, steve, sandvoss, young, mormon, arriv, lo, angel, three, fellow, missionari, evang...","[aaron, davis, steve, sandvoss, young, mormon, arrives, los, angeles, three, fellow, missionary,..."


In [32]:
val_data['msg_lemmatized']=val_data['no_stopwords'].apply(lambda x:lemmatiser(x))
val_data.head()

,title,description,clean_msg,msg_lower,msg_tokenied,no_stopwords,msg_lemmatized
0,The Fall,The psychological thriller examines the lives of two hunters -- one is a serial killer who preys...,The psychological thriller examines the lives of two hunters one is a serial killer who preys o...,the psychological thriller examines the lives of two hunters one is a serial killer who preys o...,"[the, psychological, thriller, examines, the, lives, of, two, hunters, one, is, a, serial, kille...","[psychological, thriller, examines, lives, two, hunters, one, serial, killer, preys, victims, ar...","[psychological, thriller, examines, life, two, hunter, one, serial, killer, prey, victim, around..."
1,First Love,A psychologist decides to write a book on a university student who suddenly gave up her dreams t...,A psychologist decides to write a book on a university student who suddenly gave up her dreams t...,a psychologist decides to write a book on a university student who suddenly gave up her dreams t...,"[a, psychologist, decides, to, write, a, book, on, a, university, student, who, suddenly, gave, ...","[psychologist, decides, write, book, university, student, suddenly, gave, dreams, become, announ...","[psychologist, decides, write, book, university, student, suddenly, gave, dream, become, announc..."
2,The Vow,"Following the experiences of people deeply involved in the self-improvement group NXIVM, an orga...",Following the experiences of people deeply involved in the selfimprovement group NXIVM an organi...,following the experiences of people deeply involved in the selfimprovement group nxivm an organi...,"[following, the, experiences, of, people, deeply, involved, in, the, selfimprovement, group, nxi...","[following, experiences, people, deeply, involved, selfimprovement, group, nxivm, organization, ...","[following, experience, people, deeply, involved, selfimprovement, group, nxivm, organization, s..."
3,Resident Evil,"Year 2036 -- 14 years after the spread of Joy caused so much pain, Jade Wesker fights for surviv...",Year 2036 14 years after the spread of Joy caused so much pain Jade Wesker fights for survival ...,year 2036 14 years after the spread of joy caused so much pain jade wesker fights for survival ...,"[year, 2036, 14, years, after, the, spread, of, joy, caused, so, much, pain, jade, wesker, fight...","[year, 2036, 14, years, spread, joy, caused, much, pain, jade, wesker, fights, survival, world, ...","[year, 2036, 14, year, spread, joy, caused, much, pain, jade, wesker, fight, survival, world, ov..."
4,Constantine,Demon hunter and master of the occult John Constantine figured there was no longer any point in ...,Demon hunter and master of the occult John Constantine figured there was no longer any point in ...,demon hunter and master of the occult john constantine figured there was no longer any point in ...,"[demon, hunter, and, master, of, the, occult, john, constantine, figured, there, was, no, longer...","[demon, hunter, master, occult, john, constantine, figured, longer, point, fighting, good, fight...","[demon, hunter, master, occult, john, constantine, figured, longer, point, fighting, good, fight..."


In [33]:
test_data['msg_lemmatized']=test_data['no_stopwords'].apply(lambda x:lemmatiser(x))
test_data.head()

,title,description,clean_msg,msg_lower,msg_tokenied,no_stopwords,msg_lemmatized
0,Splitting Up Together,Ellen DeGeneres serves as executive producer of this comedy that is based on a Danish series of ...,Ellen DeGeneres serves as executive producer of this comedy that is based on a Danish series of ...,ellen degeneres serves as executive producer of this comedy that is based on a danish series of ...,"[ellen, degeneres, serves, as, executive, producer, of, this, comedy, that, is, based, on, a, da...","[ellen, degeneres, serves, executive, producer, comedy, based, danish, series, name, lena, marti...","[ellen, degeneres, serf, executive, producer, comedy, based, danish, series, name, lena, martin,..."
1,The Trip,A collage of film images and ambient dance sounds from Jacques Peretti and DJ Downfall.,A collage of film images and ambient dance sounds from Jacques Peretti and DJ Downfall,a collage of film images and ambient dance sounds from jacques peretti and dj downfall,"[a, collage, of, film, images, and, ambient, dance, sounds, from, jacques, peretti, and, dj, dow...","[collage, film, images, ambient, dance, sounds, jacques, peretti, dj, downfall]","[collage, film, image, ambient, dance, sound, jacques, peretti, dj, downfall]"
2,The Kids Are Alright,"Set in the 1970s, this ensemble comedy follows a traditional Irish-Catholic family, the Clearys,...",Set in the 1970s this ensemble comedy follows a traditional IrishCatholic family the Clearys as ...,set in the 1970s this ensemble comedy follows a traditional irishcatholic family the clearys as ...,"[set, in, the, 1970s, this, ensemble, comedy, follows, a, traditional, irishcatholic, family, th...","[set, 1970s, ensemble, comedy, follows, traditional, irishcatholic, family, clearys, navigates, ...","[set, 1970s, ensemble, comedy, follows, traditional, irishcatholic, family, clearys, navigates, ..."
3,Breeders,"""Breeders"" explores the paradox experienced by nearly all parents, the willingness to die for on...",Breeders explores the paradox experienced by nearly all parents the willingness to die for ones ...,breeders explores the paradox experienced by nearly all parents the willingness to die for ones ...,"[breeders, explores, the, paradox, experienced, by, nearly, all, parents, the, willingness, to, ...","[breeders, explores, paradox, experienced, nearly, parents, willingness, die, ones, children, co...","[breeder, explores, paradox, experienced, nearly, parent, willingness, die, one, child, coupled,..."
4,Let the Right One In,"The lives of Mark and his daughter, Eleanor, were forever changed 10 years earlier when she was ...",The lives of Mark and his daughter Eleanor were forever changed 10 years earlier when she was tu...,the lives of mark and his daughter eleanor were forever changed 10 years earlier when she was tu...,"[the, lives, of, mark, and, his, daughter, eleanor, were, forever, changed, 10, years, earlier, ...","[lives, mark, daughter, eleanor, forever, changed, 10, years, earlier, turned, vampire, eleanor,...","[life, mark, daughter, eleanor, forever, changed, 10, year, earlier, turned, vampire, eleanor, a..."


**Vocabulary list**

In [34]:
vocabulary = set()      # set functions help in creating unique list
for index, row in data.iterrows():    # help in iterate over data where index is index number and row is dictionary of data
    vocabulary.update(row['msg_lemmatized'])   # updating list of vocabulary by loading tokens with each loop
# Iterate over the test dataset
for index, row in val_data.iterrows():
    vocabulary.update(row['msg_lemmatized'])
vocabulary = sorted(vocabulary)  # Sort to maintain a consistent order
vocab_size = len(vocabulary) # total number of tokens

# **TF-IDF algorithm implementation**

In [35]:
import numpy as np

def calculate_tf_idf(data):
        # Create a dictionary to map each term to an index
    term_index = {term: idx for idx, term in enumerate(vocabulary)}   # simple indexing of vocabulary to convert it to dictionary

    #  Calculate TF
    tf_matrix = []      # list defined
    for index, row in data.iterrows():
        term_freqs = {term: 0 for term in vocabulary}  # defined a dictionary with all tokens in it NOTE it will be defined again with each loop over each movie
        for term in row['msg_lemmatized']:   # current movie tokens as terms
            if term in vocabulary:  # Check if the term is in the vocabulary
              term_freqs[term] += 1    # inceasing value in dictionary

        tf_row = np.zeros(vocab_size)   # array of zero of size vocab_size
        for term, freq in term_freqs.items(): # opening dictionary to iterate over each term
            if freq > 0:    # condition if there is a token
                tf_row[term_index[term]] = np.log10(freq) + 1    #Value of term frequency is calculated using frequency and stored in tf_row at index corresponding to term whose index can be found using term_index

        tf_matrix.append(tf_row)    # for each movie an array of tf_row is appended in tf_matrix

    tf_matrix = np.array(tf_matrix)    # tf_matrix is list converted to numpy array

    # Step 3: Calculate IDF
    doc_freqs = np.zeros(vocab_size)  # defined numpy array of vocabulary size to input docuent frequency for each token
    num_docs = len(data)   # num_docs is total number of movies
    for index, row in data.iterrows():
        terms_set = set(row['msg_lemmatized'])  # Convert list of terms to a set to eliminate duplicates
        for term in terms_set:    # difference between this set and above set is this one has tokens from one movie while first one had vacabulary tokens
            if term in vocabulary:  # Check if the term is in the vocabulary
              doc_freqs[term_index[term]] += 1       # increases document frequency by one taking index from term_index and NOTE doc_freq is defined outside loop so this thing does not reset after each movie, one is increased for specific term

    idf_vector = np.zeros(vocab_size)
    for i in range(vocab_size):
        df = doc_freqs[i]
        idf_vector[i] = np.log10(num_docs / df)  if df != 0 else 0  # Using the formula log( N / 1 + df )  and frequency not equal to 0

    # Step 4: Calculate TF-IDF
    tf_idf_matrix = tf_matrix * idf_vector  # Element-wise multiplication

    return tf_idf_matrix


In [36]:
tf_idf_matrix = calculate_tf_idf(data)
print("TF-IDF Matrix:\n", tf_idf_matrix)
print("vocabulary:\n", vocabulary)

TF-IDF Matrix:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
vocabulary:
 ['', '00', '007', '007s', '1', '10', '100', '1000', '10000', '100000', '100th', '100yearold', '101st', '102yearold', '10day', '10episode', '10part', '10speed', '10yearold', '10yearoldgoingon50', '10yearolds', '11', '1100', '118', '118th', '11th', '11vignette', '11yearold', '12', '1290', '12hour', '12straight', '12th', '12thcentury', '12yearold', '13', '130', '13000', '1300mile', '1306', '13part', '13th', '13yearold', '14', '140', '1464', '147', '14th', '14yearold', '15', '150', '1500', '15091547', '150foot', '150plus', '151', '1549', '155', '1597', '15monthold', '15on', '15yearold', '16', '1603', '165', '16661', '1667', '16millimeter', '16th', '16thcentury', '16yearold', '16yearolds', '17', '1700s', '1715', '173rd', '173yearold', '1743', '175', '1770', '17month', '17th', '17thcentury', '17yearold', '18',

In [37]:
tf_idf_matrix.shape

(5968, 26200)

In [38]:
tf_idf_matrix_val = calculate_tf_idf(val_data)
print("TF-IDF Matrix:\n", tf_idf_matrix)
print("vocabulary:\n", vocabulary)

TF-IDF Matrix:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
vocabulary:
 ['', '00', '007', '007s', '1', '10', '100', '1000', '10000', '100000', '100th', '100yearold', '101st', '102yearold', '10day', '10episode', '10part', '10speed', '10yearold', '10yearoldgoingon50', '10yearolds', '11', '1100', '118', '118th', '11th', '11vignette', '11yearold', '12', '1290', '12hour', '12straight', '12th', '12thcentury', '12yearold', '13', '130', '13000', '1300mile', '1306', '13part', '13th', '13yearold', '14', '140', '1464', '147', '14th', '14yearold', '15', '150', '1500', '15091547', '150foot', '150plus', '151', '1549', '155', '1597', '15monthold', '15on', '15yearold', '16', '1603', '165', '16661', '1667', '16millimeter', '16th', '16thcentury', '16yearold', '16yearolds', '17', '1700s', '1715', '173rd', '173yearold', '1743', '175', '1770', '17month', '17th', '17thcentury', '17yearold', '18',

In [39]:
tf_idf_matrix_val.shape

(74, 26200)

In [41]:
tf_idf_matrix_test.shape

(10, 26200)

In [42]:
vocab_size

26200

# Cosine Similarity

**Cosine Similarity Function**

In [43]:
def cosine_similarity_manual(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0.0
    return dot_product / (norm_vec1 * norm_vec2)


**Calculate Similarity Matrix**

In [44]:
similarity_matrix = np.zeros((len(tf_idf_matrix_val), len(tf_idf_matrix)))

for i, val_vec in enumerate(tf_idf_matrix_val):
    for j, train_vec in enumerate(tf_idf_matrix):
        similarity_matrix[i, j] = cosine_similarity_manual(val_vec, train_vec)


In [45]:
similarity_matrix.shape

(74, 5968)

# Retrieve Top Matches

In [46]:
# Define a list to store predictions for each description
all_predictions = []

# Retrieve and store the top matches for each movie in the test dataset
for i, row in enumerate(similarity_matrix):
    # Get indices of top matches sorted by similarity score
    top_indices = row.argsort()[-3:][::-1]  # Adjust 3 to the number of top matches you want to retrieve

    # Initialize a list to store predictions for the current description
    description_predictions = []

    for idx in top_indices:
        similarity_score = row[idx]
        movie_title = train_data.iloc[idx]['title']
        description_predictions.append((movie_title, similarity_score))

    # Append the predictions for the current description to the list of all predictions
    all_predictions.append(description_predictions)

# Print the list of all predictions
for i, predictions in enumerate(all_predictions):
    print(f"Predictions for Test Movie {i + 1}:")
    print(val_data.iloc[i]['title'])
    for j, (movie_title, similarity_score) in enumerate(predictions):
        print(f"{j + 1}. Movie: {movie_title}, Similarity Score: {similarity_score}")
    print()


Predictions for Test Movie 1:
The Fall
1. Movie: Memoirs of a Murderer, Similarity Score: 0.1412776072907006
2. Movie: Lincoln Rhyme: Hunt for the Bone Collector, Similarity Score: 0.13055247122855038
3. Movie: Blind, Similarity Score: 0.12966273261665623

Predictions for Test Movie 2:
First Love
1. Movie: Fresh Meat, Similarity Score: 0.23112349206564065
2. Movie: Hush, Similarity Score: 0.20985180173889678
3. Movie: 303, Similarity Score: 0.16409438631484702

Predictions for Test Movie 3:
The Vow
1. Movie: The Marked Heart, Similarity Score: 0.1450830458447222
2. Movie: La Reina del Sur, Similarity Score: 0.13195590017984363
3. Movie: Ishqiya, Similarity Score: 0.10055096172517651

Predictions for Test Movie 4:
Resident Evil
1. Movie: Wolf, Similarity Score: 0.14229581099429317
2. Movie: Manichithrathazhu, Similarity Score: 0.10886664447978828
3. Movie: The Railway Man, Similarity Score: 0.10502234992812742

Predictions for Test Movie 5:
Constantine
1. Movie: Crazyhead, Similarity Sc

In [47]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming you have all_predictions defined
# all_predictions = ...

# Extract the top predicted movie title for each description
predicted_labels = [description_predictions[0][0] for description_predictions in all_predictions]

# Use test_data['title'] as the ground truth
ground_truth = val_data['title'].tolist()

# Calculate precision, recall, and F1 score
precision = precision_score(ground_truth, predicted_labels, average='micro')
recall = recall_score(ground_truth, predicted_labels, average='micro')
f1 = f1_score(ground_truth, predicted_labels, average='micro')

# Print the performance metrics
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Precision: 0.05405405405405406
Recall: 0.05405405405405406
F1 Score: 0.05405405405405406


Test Data

In [48]:
vocabulary = set()      # set functions help in creating unique list
for index, row in data.iterrows():    # help in iterate over data where index is index number and row is dictionary of data
    vocabulary.update(row['msg_lemmatized'])   # updating list of vocabulary by loading tokens with each loop
# Iterate over the test dataset
for index, row in test_data.iterrows():
    vocabulary.update(row['msg_lemmatized'])
vocabulary = sorted(vocabulary)  # Sort to maintain a consistent order
vocab_size = len(vocabulary) # total number of tokens

In [49]:
tf_idf_matrix_test = calculate_tf_idf(test_data)
print("TF-IDF Matrix:\n", tf_idf_matrix_test)

TF-IDF Matrix:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [52]:
tf_idf_matrix_test.shape

(10, 26065)

In [50]:
tf_idf_matrix = calculate_tf_idf(data)
print("TF-IDF Matrix:\n", tf_idf_matrix)
print("vocabulary:\n", vocabulary)

TF-IDF Matrix:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
vocabulary:
 ['', '00', '007', '007s', '1', '10', '100', '1000', '10000', '100000', '100th', '100yearold', '101st', '102yearold', '10day', '10episode', '10part', '10speed', '10yearold', '10yearoldgoingon50', '10yearolds', '11', '1100', '118', '118th', '11th', '11vignette', '11yearold', '12', '1290', '12hour', '12straight', '12th', '12thcentury', '12yearold', '13', '130', '13000', '1300mile', '1306', '13part', '13th', '13yearold', '14', '140', '1464', '147', '14th', '14yearold', '15', '150', '1500', '15091547', '150foot', '150plus', '151', '1549', '155', '1597', '15monthold', '15on', '15yearold', '16', '1603', '165', '16661', '1667', '16millimeter', '16th', '16thcentury', '16yearold', '16yearolds', '17', '1700s', '1715', '173rd', '173yearold', '1743', '175', '1770', '17month', '17th', '17thcentury', '17yearold', '18',

In [53]:
tf_idf_matrix.shape

(5968, 26065)

In [56]:
similarity_matrix = np.zeros((len(tf_idf_matrix_test), len(tf_idf_matrix)))

for i, test_vec in enumerate(tf_idf_matrix_test):
    for j, train_vec in enumerate(tf_idf_matrix):
        similarity_matrix[i, j] = cosine_similarity_manual(test_vec, train_vec)


In [57]:
# Define a list to store predictions for each description
all_predictions = []

# Retrieve and store the top matches for each movie in the test dataset
for i, row in enumerate(similarity_matrix):
    # Get indices of top matches sorted by similarity score
    top_indices = row.argsort()[-3:][::-1]  # Adjust 3 to the number of top matches you want to retrieve

    # Initialize a list to store predictions for the current description
    description_predictions = []

    for idx in top_indices:
        similarity_score = row[idx]
        movie_title = train_data.iloc[idx]['title']
        description_predictions.append((movie_title, similarity_score))

    # Append the predictions for the current description to the list of all predictions
    all_predictions.append(description_predictions)

# Print the list of all predictions
for i, predictions in enumerate(all_predictions):
    print(f"Predictions for Test Movie {i + 1}:")
    print(val_data.iloc[i]['title'])
    for j, (movie_title, similarity_score) in enumerate(predictions):
        print(f"{j + 1}. Movie: {movie_title}, Similarity Score: {similarity_score}")
    print()


Predictions for Test Movie 1:
The Fall
1. Movie: Happily Divorced, Similarity Score: 0.16344973027907225
2. Movie: Colin from Accounts, Similarity Score: 0.12064271203526543
3. Movie: Hawaii, Similarity Score: 0.10567986736738277

Predictions for Test Movie 2:
First Love
1. Movie: Mesrine: Killer Instinct, Similarity Score: 0.15496101517927108
2. Movie: The Ellen DeGeneres Show, Similarity Score: 0.1135847493874442
3. Movie: Kerala Varma Pazhassi Raja, Similarity Score: 0.1120984982699343

Predictions for Test Movie 3:
The Vow
1. Movie: Bob Biswas, Similarity Score: 0.16557831199592
2. Movie: The Guest, Similarity Score: 0.14974194602931437
3. Movie: The Great Indian Kitchen, Similarity Score: 0.1405753686517587

Predictions for Test Movie 4:
Resident Evil
1. Movie: #BlackAF, Similarity Score: 0.18169546707218254
2. Movie: Summerland, Similarity Score: 0.10663822551160228
3. Movie: Two Is a Family, Similarity Score: 0.10569604680907281

Predictions for Test Movie 5:
Constantine
1. Movi

In [59]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming you have all_predictions defined
# all_predictions = ...

# Extract the top predicted movie title for each description
predicted_labels = [description_predictions[0][0] for description_predictions in all_predictions]

# Use test_data['title'] as the ground truth
ground_truth = test_data['title'].tolist()

# Calculate precision, recall, and F1 score
precision = precision_score(ground_truth, predicted_labels, average='micro')
recall = recall_score(ground_truth, predicted_labels, average='micro')
f1 = f1_score(ground_truth, predicted_labels, average='micro')

# Print the performance metrics
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Precision: 0.0
Recall: 0.0
F1 Score: 0.0
